In [ ]:
import scipy.signal as signal
import numpy as np
import numpy.fft as fft
from numpy.random import randn
import matplotlib.pyplot as plt
import timeit

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,6

In [ ]:
def time_conv(conv, nu, nv):
    u = randn(nu) + 1j * randn(nu)
    v = randn(nv) + 1j * randn(nv)
    t = %timeit -q -o -r 2 conv(u, v)
    return t.average

In [ ]:
def benchmark_conv(conv, sizes_h, sizes_x):
    times = np.zeros((len(sizes_h), len(sizes_x)))
    for i in range(len(sizes_h)):
        for j in range(len(sizes_x)):
            size_h = sizes_h[i]
            size_x = sizes_x[j]
            if size_h > size_x:
                times[i,j] = np.nan
            else:
                time = time_conv(conv, size_x, size_h)
                times[i,j] = time * 1e3
#                 print("({},{}) -- {:0.2f} ms".format(size_h, size_x, time * 1e3))
    return times

In [ ]:
def osconvolve(in1, in2):
    # Inputs:  1-D numpy array, in1, an input signal
    #          1-D numpy array, in2, an input signal
    # Outputs: 1-D numpy array, y, the output of convolving in1 and in2
    
    # We only want 1-D convolution
    assert len(in1.shape) == 1
    assert len(in2.shape) == 1
    
    # Assume the shorter input is the filter h, and the longer input is the signal x
    h = in1 if len(in1) < len(in2) else in2 # filter (shorter)
    x = in2 if len(in1) < len(in2) else in1 # signal (longer)
    
    P = len(h) # filter length
    y = np.zeros(???, dtype=x.dtype) # need to force dtype for complex values
    
    nfft = ??? # FFT length
    L = ??? # number of new input samples per input block
    
    # Compute and store DFT of filter
    H = ??? # H is nfft-length DFT of filter h
    
    # Initialize nfft-length block arrays
    ???
    
    # Iterate over blocks
    ???
    
    return y # final convolution output!

In [ ]:
# check output for correctness
size_x = 2 ** 10
size_h = 2 ** 5
x = randn(size_x) + 1j * randn(size_x)
h = randn(size_h) * 1j * randn(size_h)
y = osconvolve(h, x)
y0 = signal.convolve(h, x)
rmse = np.sqrt( (abs(y - y0) ** 2).mean() )
print("RMSE = ", rmse)

In [ ]:
# benchmark functions (SLOW! May take a few minutes on the Pi.)
sizes_h = 2 ** np.array([4,8,12])
sizes_x = 2 ** np.array([4,6,8,10,12,14])
times_dir = benchmark_conv(np.convolve, sizes_h, sizes_x)
times_fft = benchmark_conv(signal.fftconvolve, sizes_h, sizes_x)
times_ola = benchmark_conv(signal.oaconvolve, sizes_h, sizes_x)
times_ols = benchmark_conv(osconvolve, sizes_h, sizes_x)
times_sci = benchmark_conv(signal.convolve, sizes_h, sizes_x)

In [ ]:
# plot benchmark data
for i in range(len(sizes_h)):
    fig, ax = plt.subplots()
    plt.loglog(sizes_x, times_dir[i], '.-')
    plt.loglog(sizes_x, times_fft[i], '.-')
    plt.loglog(sizes_x, times_ola[i], '.-')
    plt.loglog(sizes_x, times_ols[i], '.-')
    plt.loglog(sizes_x, times_sci[i], '.-')
    plt.legend(["Direct", "FFT", "OLA", "OLS (mine)", "Scipy's choice"])
    plt.title("Filter size = {}".format(sizes_h[i]))
    plt.xlabel("Input signal size")
    plt.ylabel("Time [ms]")
    plt.show()

#### Some questions:
1. When convolving a signal with a very short filter, which convolution method is generally the fastest?
2. When convolving two very long signals, which convolution method is generally the fastest?
3. When convolving a very, very long signal with a long (but not as long) filter, which convolution method is generally the fastest?
4. How does your overlap-save implementation compare to Scipy's different implementations?
5. The function scipy.convolve chooses between np.convolve, signal.fftconvolve, and signal.oaconvolve, depending on what it thinks will be the fastest after computing some heuristics based on historical benchmarks. How does it perform?

#### Your answers here:
